<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Analysis**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

2.2.0


## **Variationsrechnung**

Content

## **Funktionentheorie**

Content

## **Vektoranalysis**

Content

## **Fourier-Analysis**

Content

## **Differentialformen**

Content

## **Differentialgeometrie**

Content